In [2]:
import os
os.environ["OMP_NUM_THREADS"] = "32"

In [3]:
from graph_tool.all import *
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.covariance import LedoitWolf, OAS
import matplotlib.pyplot as plt
import matplotlib.cm as mpl


import statsmodels.api as sm
from multipy.fdr import qvalue
from multipy.fdr import lsu

import dill

import fit_sbm
import trim_networks
import export_blocks

In [5]:
 g = load_graph("../data/output/SBM/gtex/trimmed_graph/fdr-1e-5/14_TESTIS.xml.gz")

In [6]:
g

<Graph object, undirected, with 5917 vertices and 5030767 edges, 2 internal vertex properties, 2 internal edge properties, at 0x7f020d190490>

In [8]:
N = len(g.get_vertices())
print(str(N) + " genes")
Et = (N * N - N)/2
E = len(g.get_edges())
density = E/Et        
print("Density: " + str(density))

5917 genes
Density: 0.2874315682926414


In [10]:
corr = g.edge_properties["spearman"]
corr
g.ep.z_s = g.new_edge_property("double", (2*np.arctanh(corr.a)))

In [40]:
state_min = minimize_nested_blockmodel_dl(gi, init_bs=None,
                                          state_args=dict(recs=[gi.ep.z_s],
                                          rec_types=["real-normal"]))

In [42]:
S1 = state_min.entropy()
print("Starting annealing...")
mcmc_anneal(state_min, beta_range=(1, 10), niter=100,
            mcmc_equilibrate_args=dict(force_niter=100))
S2 = state_min.entropy()
print("Improvement from annealing:", S2 - S1)
print("Final entropy after annealing: " + str(state_min.entropy()))

Starting annealing...
Improvement from annealing: -2846369.330363484
Final entropy after annealing: 14586723.511638353


In [ ]:
state_min.draw()